##INSAID Capstone project - group 1015

###Installing and importing mysql.connector/python

In [106]:
!pip install --user mysql-connector-python

In [0]:
import mysql.connector

###Creating db connection and storing in capdb

In [0]:
 capdbConfig = {
     'host' : 'cpanel.insaid.co',
     'user' : 'student',
      'password' : 'student', 
      'database' : 'Capstone1',
      'connection_timeout' : 180,
      'use_pure' : False
      #use_pure = False ensures the C extension is used for MySQL connection which is faster than pure python connection
 }
 capdb = mysql.connector.connect(**capdbConfig)

###Creating python cursor for executing mysql queries, checking tables, structures and count

In [0]:
capdbCursor = capdb.cursor()
cursor.execute("SET max_allowed_packet=1073741824") #setting max served packet size as 1 GB

In [110]:
capdbCursor.execute("show tables")
capdbTables = capdbCursor.fetchall()
for table in capdbTables:
    print(table)

('events_data',)
('gender_age_train',)
('phone_brand_device_model',)


In [111]:
capdbCursor.execute("describe events_data")
print(capdbCursor.fetchall())

[('event_id', 'bigint(20)', 'NO', '', None, ''), ('device_id', 'bigint(255)', 'YES', '', None, ''), ('timestamp', 'datetime', 'NO', '', None, ''), ('longitude', 'float(8,6)', 'YES', '', None, ''), ('latitude', 'float(8,6)', 'YES', '', None, ''), ('city', 'varchar(50)', 'NO', '', None, ''), ('state', 'varchar(50)', 'NO', '', None, '')]


In [112]:
capdbCursor.execute("select count(*) from events_data where state = 'delhi'")
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

(751733,)


In [113]:
capdbCursor.execute("describe phone_brand_device_model")
print(capdbCursor.fetchall())

[('device_id', 'bigint(255)', 'NO', '', None, ''), ('phone_brand', 'varchar(2555)', 'NO', '', None, ''), ('device_model', 'varchar(2555)', 'NO', '', None, '')]


In [114]:
query =   """select count(*)
            from phone_brand_device_model m
            left join events_data e
            on m.device_id = e.device_id
            where e.state = 'delhi'"""
capdbCursor.execute(query)
capdbRecords = capdbCursor.fetchall()
for record in capdbRecords:
    print(record)

SyntaxError: ignored

### Closing db connection

In [0]:
capdb.close()